In [1]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


In [2]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
#set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [5]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [6]:
def gen_code_table(llm, messages, question, table): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"table": table, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(table = table, question = question), code)

def gen_code_values(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return ("", code)

def exec_code_table(code, table):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({table})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')
            
def exec_code_values(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')
            


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0, top_p = 1.0, seed=11)
llm = ChatOpenAI(model="gpt-3.5-turbo", 
temperature=0,
    #max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    seed=123)

In [10]:
 messages_table=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a table. You will receive the financial report as a table and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as table: {table}"
        ),         
        (
          "ai",
          "Ok, I received the table."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
          "Generate a Python function 'run(table)' that can answer the question using the table! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "The code must be specific to the provided table. " +
            "Do not generate explanation, nor example code, just the function. "              
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]     

 messages_values=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a list of annotated values: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
          "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "The code must be specific to the provided value list. " +
            "Do not generate explanation, nor example code, just the function. "              
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]     

In [30]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

#qid = 'ba6783f3-8207-419a-b407-3f688682caef'
#qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'

#qid = 'bf7abd62-d9cd-48d2-8826-1457684019a3'
#qid = '4d259081-6da6-44bd-8830-e4de0031744c'
#qid = 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'

#qid = '7cd3aedf-1291-4fea-bc9d-a25c65727b7b'
#qid = '22e20f25-669a-46b9-8779-2768ba391955'
qid = '65ec782c-691e-45df-b541-caecb85154ff'
#qid = 'a983501d-2eec-486d-9661-e520c7c8af5e'
#qid = '6100c476-160a-4f1e-bfc1-a16f4cc18b52'
#qid = 'ef274d2b-fbd6-4e9f-95f9-79b37827d91c'
#qid = '94916f79-4b6d-487f-8fa2-39f81a085760'
#qid = '72325ec6-41ad-4648-9798-b22a61122cb4'
#qid = '4f7c8e0a-6ae9-40ee-bd16-06db6f96eaf1'
_table, _q =  get_question(qid)
table = _table['table']

[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_multitable(table)
[print(v) for v in values]
#response = chain.invoke({"value_list": values, "question":q})
#code =  response.replace('```python','').replace('```','')
p, code = gen_code_table(llm, messages_table, q, table)
print(code)
#print(len(p) + len(code))
(v, s) = exec_code_table(code, table)
rng = 0.01
print(_q['answer'])
if v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng):
    print("\033[92m Success: " + str(v)+'\033[0m')
else:    
    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  

['', '', '2019', '2018']
['', 'Note', '£m', '£m']
['Fixed assets', '', '', '']
['Investments', '3', '1,216.0', '1,212.9']
['', '', '1,216.0', '1,212.9']
['Current assets', '', '', '']
['Debtors', '4', '415.9', '440.7']
['Cash and cash equivalents', '5', '–', '0.2']
['', '', '415.9', '440.9']
['Creditors: amounts falling due within one year', '6', '(411.4)', '(288.4)']
['Net current assets', '', '4.5', '152.5']
['Net assets', '', '1,220.5', '1,365.4']
['Capital and reserves', '', '', '']
['Called-up share capital', '9', '9.3', '9.5']
['Own shares held', '10', '(16.5)', '(16.9)']
['Capital redemption reserve', '', '0.7', '0.5']
['Retained earnings', '', '1,227.0', '1,372.3']
['Total equity', '', '1,220.5', '1,365.4']
What was the change in Capital redemption reserve in 2019 from 2018?
{'number_value': 3.0, 'scale': '', 'category': 'Investments', 'header1': 'Note'}
{'number_value': 1216.0, 'scale': '', 'category': 'Investments', 'header1': '£m', 'header2': '2019'}
{'number_value': 4.0, 's

In [34]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

qids = [
#'ba6783f3-8207-419a-b407-3f688682caef',
#'a0414f81-8dc2-44b2-a441-2c9d9c805c4d',
#'bf7abd62-d9cd-48d2-8826-1457684019a3',
#'4d259081-6da6-44bd-8830-e4de0031744c',
#'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20',

#'7cd3aedf-1291-4fea-bc9d-a25c65727b7b',
#'22e20f25-669a-46b9-8779-2768ba391955',
'65ec782c-691e-45df-b541-caecb85154ff',
'a983501d-2eec-486d-9661-e520c7c8af5e',
'ef274d2b-fbd6-4e9f-95f9-79b37827d91c'
]
def get_answer(llm, messages, table, q):
    values = table_convert.convert_multitable(table)    
    if len(values) > 10000:
        print('Using value list')
        p, code = gen_code_values(llm, messages_values, q, values)    
        #print(code)
        (v, s) = exec_code_values(code, values)
        return  (v, s)
    else:
        print('Using table')
        p, code = gen_code_table(llm, messages_table, q, table)    
        #print(code)
        (v, s) = exec_code_table(code, table)
        return  (v, s)



for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    
    #[print(r) for r in table]
    
    q = _q['question']
    print(q, end='')

    
    #r = []
    #for i in [0,1,2]:        
    #   r.append(get_answer(llm, messages, table, q))
    
    #if r[0][0] == r[1][0]:
    #    v = r[0][0]
    #elif r[1][0] == r[2][0]:
    #    v = r[1][0]
    #elif r[0][0] == r[2][0]:
    #    v = r[0][0] 
    #else:
    #    v = None
        
    (v, s) = get_answer(llm, messages, table, q)    
    
    if v == _q['answer']:
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 

What was the change in Capital redemption reserve in 2019 from 2018?Using table
 failure: -145.3 good answer:  0.2 
What was the average difference between EBITDA and underlying EBITDA for both FYs?Using table
 failure: -2349.0 good answer:  3728 
What was the total expenses for Oracle in 2018?Using table
 Success: 35796


In [188]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #value_list = table_convert.convert_multitable(table)    
    
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:   
            _table, _q =  get_question(q['uid'])
            
            table = _table['table']
            
            #[print(r) for r in table]
            
            q = _q['question']
            print(q, end='')
            
            
            #r = []
            #for i in [0,1,2]:        
            #   r.append(get_answer(llm, messages, table, q))
            
            #if r[0][0] == r[1][0]:
            #    (v, s) = r[0]
            #elif r[1][0] == r[2][0]:
            #    (v, s) = r[1]
            #elif r[0][0] == r[2][0]:
            #    (v, s) = r[0]
            #else:
            #    (v, s) = (None, None)
                
            (v, s) = get_answer(llm, messages, table, q)    
            
            rng = 0.01
            if not isinstance(v, str)  and ( v == _q['answer'] or v > _q['answer']-rng or v < _q['answer']+rng):
                print("\033[92m Success: " + str(v)+'\033[0m')
            else:    
                print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 
            
            
            (pred_value, pred_scale) = (v,s)
            if isinstance(pred_value, tuple) and len(pred_value) == 2:
                print('$$$$')
                (pred_value, pred_scale) = pred_value
            if pred_scale == "%" or pred_scale == "percentage"  :
                pred_scale = 'percent'
            if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                print('Invalid ', pred_scale)
                pred_scale = ""    
            #if item['table']['uid'] in res_scale and pred_scale != 'percent':
            #    pred_scale = res_scale[item['table']['uid']]
            if isinstance(pred_value, str):
                #print("string")
                if  pred_value.startswith('[Error]'):
                    (pred_value, pred_scale) = ("", "")
                    err = v                                                        
            res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, pred_value, pred_scale, _q, code, prompt,value_list, err))

0it [00:00, ?it/s]

What is the change in Other in 2019 from 2018? Success: -12.6
What is the percentage change in Other in 2019 from 2018? Success: -22.22
What was the change in the amount for Appliances in 2019 from 2018? Success: -94.0
What was the percentage change in the amount for Appliances in 2019 from 2018? Success: -7.02
What is the difference between the domestic and international discount rates as at September 30, 2019? Success: 2.1
What is the year on year percentage change in domestic discount rate between 2018 and 2019? Success: 6.67
What is the year on year percentage change in international expected return on plan assets between 2018 and 2019? Success: -8.11
What is the 2019 average defined contribution schemes? Success: 172.0
What is the 2019 average defined benefit schemes? Success: 50.5
What is the difference between 2019 average defined contribution schemes and 2019 average defined benefit schemes? Success: 121.5
What is the percentage change in the operating loss carryforward from 20

KeyboardInterrupt: 

In [189]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.4222222222222222 0.4222222222222222 0.6


In [190]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'],q['derivation'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        print(q['question'])
        #print(value_list)
        #print(value_list)
        #print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

fe11f001-3bfe-4089-8108-412676f0a780 (680-774)/774 -7.02 percent  <>  -12.14 percent
What was the percentage change in the amount for Appliances in 2019 from 2018?
bed1fce2-69cb-4d1e-a34a-01950a1770bd (73,260-57,768)/57,768 -21.15 percent  <>  26.82 percent
What is the percentage change in the operating loss carryforward from 2018 to 2019?
348d031d-73ab-4b35-af46-998cfef25775 (19,911-15,916)/15,916 -25.1 percent  <>  25.1 percent
What is the percentage change in the valuation allowance from 2018 to 2019?
aea850af-68cb-4cc1-80b2-c9c2860b273e {  } 2.98 percent [Error]could not convert string to float: '(15916)'
263d03ec-83d2-48df-8376-1a72167798f7 {  } 0.87  [Error]could not convert string to float: '$3025'
78fc6d55-c20c-4f71-99fe-bc40a16e61d0 2,664 - 3,025  361.0 million  <>  -361 
What is the change of IMFT’s total assets from 2018 to 2019?
dc5e217a-a7b3-4fc9-ac0f-13d328f26b20 {  } 4227.5 million [Error]unterminated string literal (detected at line 4) (<string>, line 4)
22e20f25-669a-4